In [ ]:
# Ada Ceylin Cengiz
# 2022211045

In [ ]:
# question 1

In [1]:
def get_page(url):
  try:
    import urllib.request
    page = urllib.request.urlopen(url).read()
    page = page.decode("utf-8")
    return page
  except:
    return ""


def get_next_target(page):
    start_link = page.find('<a href=')
    if start_link == -1:
        return None, 0
    start_quote = page.find('"', start_link)
    end_quote = page.find('"', start_quote+1)
    url = page[start_quote + 1:end_quote]
    return url, end_quote


def get_all_links(page):
    links = []
    while True:
      url, endpos = get_next_target(page)
      if url:
        links.append(url)
        page = page[endpos:]
      else:
        break
    return links


def union(p,q):
    for e in q:
        if e not in p:
            p.append(e)


def add_toIndex(index, keyword, url):
    if keyword in index:
        index[keyword].append(url)
    else:
        index[keyword] = [url]


def getclearpage(content):
  title = content[content.find("<title>")+7:content.find("</title>")]
  body = content[content.find("<body>")+6:content.find("</body>")]
  while body.find(">") != -1:
    start =  body.find("<")
    end =  body.find(">")
    body = body[:start] + body[end+1:]
  return title + body


def addPageToIndex(index, url, content):
  content = getclearpage(content)
  words = content.split()
  for word in words:
    add_toIndex(index, word, url)


def crawlWeb(seed):
  tocrawl = [seed]
  crawled = []
  index = {}
  graph = {}
  while tocrawl:
    page = tocrawl.pop()
    if page not in crawled:
      content = get_page(page)
      addPageToIndex(index, page, content)
      outlinks = get_all_links(content)
      graph[page] = outlinks
      union(tocrawl, get_all_links(content))
      crawled.append(page)
  return index, graph

# The returned graph is a directed graph where
# each URL acts as a node and is mapped to a list of URLs,
# forming the edges. The graph represents the interconnected
# structure of the web.


def lookup(index, keyword):
    if keyword in index:
        return index[keyword]
    else:
        return None

index, graph = crawlWeb("https://searchengineplaces.com.tr/")


In [2]:
# 1b

output = f"The graph has {len(graph)} elements. These are:\n"

index = 1
for page in graph:
    outlinks = graph[page]
    output += f"\t{index}. [{page}]:{outlinks}\n"
    index += 1

print(output)


The graph has 10 elements. These are:
	1. [https://searchengineplaces.com.tr/]:['http://www.searchengineplaces.com.tr/travel_guide.html']
	2. [http://www.searchengineplaces.com.tr/travel_guide.html]:['http://www.searchengineplaces.com.tr/ankara.html', 'http://www.searchengineplaces.com.tr/konya.html', 'http://www.searchengineplaces.com.tr/istanbul.html', 'http://www.searchengineplaces.com.tr/oktayrecommends.html', 'http://www.searchengineplaces.com.tr/seymarecommends.html']
	3. [http://www.searchengineplaces.com.tr/seymarecommends.html]:['http://www.searchengineplaces.com.tr/oktayrecommends.html', 'http://www.searchengineplaces.com.tr/konya.html']
	4. [http://www.searchengineplaces.com.tr/oktayrecommends.html]:['http://www.searchengineplaces.com.tr/istanbul.html']
	5. [http://www.searchengineplaces.com.tr/istanbul.html]:['http://www.searchengineplaces.com.tr/maidens_tower.html', 'http://www.searchengineplaces.com.tr/galata_tower.html']
	6. [http://www.searchengineplaces.com.tr/galata_t

In [3]:
# 1c
def computeRanks(graph):
  d = 0.8
  N = len(graph)
  numloops = 10
  ranks = {}
  for page in graph:
    ranks[page] = 1/N

  for i in range(0, numloops):
    newranks = {}
    for page in graph:
      newrank = (1-d)/N
      for node in graph:
        if page in graph[node]:
          newrank = newrank + d*(ranks[node]/len(graph[node]))
      newranks[page] = newrank
    ranks = newranks
  return newranks


In [4]:
index , graph = crawlWeb ("https://searchengineplaces.com.tr/")
ranks = computeRanks ( graph )
print ( ranks )

{'https://searchengineplaces.com.tr/': 0.019999999999999997, 'http://www.searchengineplaces.com.tr/travel_guide.html': 0.14780429869056003, 'http://www.searchengineplaces.com.tr/seymarecommends.html': 0.073060618584064, 'http://www.searchengineplaces.com.tr/oktayrecommends.html': 0.073060618584064, 'http://www.searchengineplaces.com.tr/istanbul.html': 0.17460832634470402, 'http://www.searchengineplaces.com.tr/galata_tower.html': 0.09025810358272002, 'http://www.searchengineplaces.com.tr/maidens_tower.html': 0.09025810358272002, 'http://www.searchengineplaces.com.tr/konya.html': 0.073060618584064, 'http://www.searchengineplaces.com.tr/mevlana.html': 0.04928445079552, 'http://www.searchengineplaces.com.tr/ankara.html': 0.043776167788544}


In [5]:
ranks = computeRanks(graph)

output = ""
for page in ranks:
    rank = ranks[page]
    output += f"The rank of the page [{page}]: {rank}\n"
output

print(output)

The rank of the page [https://searchengineplaces.com.tr/]: 0.019999999999999997
The rank of the page [http://www.searchengineplaces.com.tr/travel_guide.html]: 0.14780429869056003
The rank of the page [http://www.searchengineplaces.com.tr/seymarecommends.html]: 0.073060618584064
The rank of the page [http://www.searchengineplaces.com.tr/oktayrecommends.html]: 0.073060618584064
The rank of the page [http://www.searchengineplaces.com.tr/istanbul.html]: 0.17460832634470402
The rank of the page [http://www.searchengineplaces.com.tr/galata_tower.html]: 0.09025810358272002
The rank of the page [http://www.searchengineplaces.com.tr/maidens_tower.html]: 0.09025810358272002
The rank of the page [http://www.searchengineplaces.com.tr/konya.html]: 0.073060618584064
The rank of the page [http://www.searchengineplaces.com.tr/mevlana.html]: 0.04928445079552
The rank of the page [http://www.searchengineplaces.com.tr/ankara.html]: 0.043776167788544



In [7]:
# 1d
def rankedLookup(index, key, graph):
    ranks = computeRanks(graph)

    if key not in index:
        return []
    pages = list(set(index[key]))
    def get_rank(page):
        return ranks[page]
    sorted_pages = sorted(pages, key=get_rank, reverse=True)

    return sorted_pages

In [8]:
results = rankedLookup ( index , "in", graph )
for result in results :
  print ( result )

http://www.searchengineplaces.com.tr/istanbul.html
http://www.searchengineplaces.com.tr/travel_guide.html
http://www.searchengineplaces.com.tr/galata_tower.html
http://www.searchengineplaces.com.tr/maidens_tower.html
http://www.searchengineplaces.com.tr/konya.html
http://www.searchengineplaces.com.tr/mevlana.html


In [11]:
# 1e
def lookup(index, key, *args):


    warning_message = ("""This procedure takes 4 outputs these are:
1-An index
2-A key
3-A graph
4-A computing procedure respectively.
You have 2 option to use this lookup procedure: with or without ranking.
-If you intented to use it without page rank be sure you have given two inputs, index and key, respectively.
-If you intented to use it with page rank be sure you have given all four input in the given order.
INVALID INPUT COMBINATION: Please check the inputs."""
    )

    if len(args) == 0:
        if key in index:
            return list(set(index[key]))
        return []


    elif len(args) == 1:
        print(warning_message)

    elif len(args) == 2:
      graph, compute_ranks = args
      if key not in index:
        return []
      return rankedLookup(index, key, graph)


In [15]:
# Test1

see = lookup ( index , "in", graph , computeRanks ) # ATTENTION !!!!!
for e in see :
  print ( e )

http://www.searchengineplaces.com.tr/istanbul.html
http://www.searchengineplaces.com.tr/travel_guide.html
http://www.searchengineplaces.com.tr/galata_tower.html
http://www.searchengineplaces.com.tr/maidens_tower.html
http://www.searchengineplaces.com.tr/konya.html
http://www.searchengineplaces.com.tr/mevlana.html


In [18]:
# Test for duplicate outputs

assert rankedLookup( index , "in", graph ) == lookup( index , "in", graph , computeRanks )

In [17]:
# Test 2

see1 = lookup ( index , "in")
for e in see1 :
  print(e)

http://www.searchengineplaces.com.tr/istanbul.html
http://www.searchengineplaces.com.tr/konya.html
http://www.searchengineplaces.com.tr/galata_tower.html
http://www.searchengineplaces.com.tr/mevlana.html
http://www.searchengineplaces.com.tr/maidens_tower.html
http://www.searchengineplaces.com.tr/travel_guide.html


In [13]:
# Test 3

lookup ( index , "in", graph )

This procedure takes 4 outputs these are:
1-An index
2-A key
3-A graph
4-A computing procedure respectively.
You have 2 option to use this lookup procedure: with or without ranking.
-If you intented to use it without page rank be sure you have given two inputs, index and key, respectively.
-If you intented to use it with page rank be sure you have given all four input in the given order.
INVALID INPUT COMBINATION: Please check the inputs.


In [14]:
# Test 4

lookup ( index , "in", computeRanks )

This procedure takes 4 outputs these are:
1-An index
2-A key
3-A graph
4-A computing procedure respectively.
You have 2 option to use this lookup procedure: with or without ranking.
-If you intented to use it without page rank be sure you have given two inputs, index and key, respectively.
-If you intented to use it with page rank be sure you have given all four input in the given order.
INVALID INPUT COMBINATION: Please check the inputs.


In [ ]:
# question 2

In [19]:
import requests
import random

def download_file_from_drive(file_url, colab_path):
    if "drive.google.com" in file_url:
        file_id = file_url.split('/d/')[1].split('/')[0]
        download_url = f"https://drive.google.com/uc?export=download&id={file_id}"
    else:
        raise ValueError("The provided URL is not a valid Google Drive link.")

    response = requests.get(download_url, stream=True)

    if response.status_code == 200:
        with open(colab_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    file.write(chunk)
        print(f"File successfully downloaded to {colab_path}")
    else:
        raise Exception("Failed to download the file. Check the URL or file permissions.")

colab_path = "/content/words.txt"
file_url = "https://drive.google.com/file/d/1cjCZ-5uAHVFLXLV6HuSM98_QblWaXrBZ/view?usp=share_link"
download_file_from_drive(file_url,colab_path)


WORDLIST_FILENAME = "words.txt"

def load_words():

  inFile = open(WORDLIST_FILENAME, 'r')
  line = inFile.readline()
  wordlist = line.split()
  print("  ", len(wordlist), "words loaded.")
  return wordlist

def word_select(wordlist, secretword_length):
  return random.choice([word for word in wordlist if len(word) == secretword_length])

def updatelist1(secretword, guess):
  return [guess[i] if guess[i] == secretword[i] else None for i in range(len(secretword))]

def updatelist2(secretword, guess):
  return [guess[i] for i in range(len(secretword)) if guess[i] in secretword and guess[i] != secretword[i]]

def updatelist3(secretword, guess):
  return [letter for letter in guess if letter not in secretword]

def cleanwordlist(wordlist, list1, list2, list3):

    valid_words = []

    for word in wordlist:
        is_valid = True

        for i in range(len(list1)):
            if list1[i] is not None and word[i] != list1[i]:
                is_valid = False
                break

        if is_valid:
            for letter in list2:
                if letter not in word:
                    is_valid = False
                    break

        if is_valid:
            for letter in list3:
                if letter in word:
                    is_valid = False
                    break

        if is_valid:
            valid_words.append(word)

    return valid_words

def wordle(secretword, guess):
  list1 = updatelist1(secretword,guess)
  list2 = updatelist2(secretword,guess)
  list3 = updatelist3(secretword,guess)
  return list1, list2, list3

def generateguess(wordlist):
  return random.choice(wordlist)

def optimize_wordle():
    wordlist = load_words()
    secretword = random.choice(wordlist)
    secretword_length = len(secretword)
    wordlist = [word for word in wordlist if len(word) == secretword_length]

    attempts = 0
    used_guesses = set()
    guesses = []

    guess = word_select(wordlist, secretword_length)

    while True:
        attempts += 1
        guesses.append(guess)
        used_guesses.add(guess)

        list1, list2, list3 = wordle(secretword, guess)

        if guess == secretword:
            print(f"Secret word is: {secretword}\n")
            print("Computer will try to find this word...\n")
            for i in range(len(guesses)):
              print(f"Computer guess is:", guesses[i])
            print(f"\nThe computer guessed the word in {attempts} attempts.\n")
            print(f"List of the computer's guesses: {guesses}")
            break

        wordlist = cleanwordlist(wordlist, list1, list2, list3)
        wordlist = [word for word in wordlist if word not in used_guesses]

        if not wordlist:
            print("No more valid words to guess.")
            break

        guess = generateguess(wordlist)

File successfully downloaded to /content/words.txt


In [20]:
optimize_wordle()

   55900 words loaded.
Secret word is: roans

Computer will try to find this word...

Computer guess is: abyss
Computer guess is: togas
Computer guess is: loams
Computer guess is: roars
Computer guess is: roans

The computer guessed the word in 5 attempts.

List of the computer's guesses: ['abyss', 'togas', 'loams', 'roars', 'roans']
